In [ ]:
import pyomo.environ as pe
import pyomo.opt as po

In [ ]:
solver = po.SolverFactory('glpk')

## Model (Specific)
Consider the following binary knapsack instance.

\begin{align}
    \text{max}~~  & 3 x_1 + 4 x_2 + 5 x_3 + 8 x_4 + 9 x_5 \\
    \text{s.t.}~~ & 2 x_1 + 3 x_2 + 4 x_3 + 7 x_4 + 9 x_5 \le 20 \\
                  & x_1, x_2, x_3, x_4, x_5 \in \{0, 1\}
\end{align}

## Implement (Specific)

In [ ]:
M = pe.ConcreteModel()

In [ ]:
M.x1 = pe.Var(domain=pe.Binary)
M.x2 = pe.Var(domain=pe.Binary)
M.x3 = pe.Var(domain=pe.Binary)
M.x4 = pe.Var(domain=pe.Binary)
M.x5 = pe.Var(domain=pe.Binary)

In [ ]:
obj_expr = 3 * M.x1 + 4 * M.x2 + 5 * M.x3 + 8 * M.x4 + 9 * M.x5
M.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [ ]:
con_expr = 2 * M.x1 + 3 * M.x2 + 4 * M.x3 + 5 * M.x4 + 9 * M.x5 <= 20
M.con = pe.Constraint(expr=con_expr)

## Solve and Postprocess

In [ ]:
result = solver.solve(M)

In [ ]:
print(M.x1.value)
print(M.x2.value)
print(M.x3.value)
print(M.x4.value)
print(M.x5.value)

In [ ]:
M.obj()

In [ ]:
pe.value(M.obj)

## Model (General)
The best way to use Pyomo is to consider the implement using the general form of the problem. The instance above is generalized by the formulation below.

\begin{align}
    \text{max}~~  & \sum_{i \in N}{c_i x_i} \\
    \text{s.t.}~~ & \sum_{i \in N}{a_i x_i} \le b \\
                  & x_i \in \{0, 1\}, \forall i \in N
\end{align}

## Implement (General)

In [ ]:
M = pe.ConcreteModel()

In [ ]:
M.N = pe.RangeSet(1, 5)

In [ ]:
print(set(M.N))

In [ ]:
c = {1: 3, 2: 4, 3: 5, 4: 8, 5: 9}
a = {1: 2, 2: 3, 3: 4, 4: 5, 5: 9}
b = 20

In [ ]:
M.c = pe.Param(M.N, initialize=c)
M.a = pe.Param(M.N, initialize=a)
M.b = pe.Param(initialize=b)

In [ ]:
print(M.c)
print(M.c[2])
print(M.b)
print(M.b.value)

In [ ]:
M.x = pe.Var(M.N, domain=pe.Binary)

In [ ]:
obj_expr = sum(M.c[i] * M.x[i] for i in M.N)
M.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [ ]:
con_expr = sum(M.a[i] * M.x[i] for i in M.N) <= M.b
M.con = pe.Constraint(expr=con_expr)

In [ ]:
result = solver.solve(M)

In [ ]:
for i in M.N:
    print(pe.value(M.x[i]), M.x[i].value)

In [ ]:
print(M.obj())

## Benefits

In [ ]:
# this example will fail and this is deliberate
# make N a subset of what we really want
M = pe.ConcreteModel()
M.N = pe.RangeSet(1, 4)
M.c = pe.Param(M.N, initialize=c) # this line fails

In [ ]:
# this example will fail and this is deliberate
# make N a superset of what we really want
inst = pe.ConcreteModel()
inst.N = pe.RangeSet(1, 20)
inst.c = pe.Param(inst.N, initialize=c) # this line fails